Overall Top-5 Accuracy

In [60]:
## CLIP package installation

In [1]:
%%capture
!conda install --yes -c pytorch pytorch=1.7.1 torchvision cudatoolkit=11.0
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

In [2]:
import torch
import clip
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)
#model is CLIP model, preproccess is an image transformation for the model

In [11]:
type(model)

In [7]:
preprocess

## Text labels for CLIP

## Downloading from Google Images 

In [12]:
import zipfile
import os

zip_file_path = '/home/ubuntu/Kate/clip_alex/4robbi.zip'
extract_folder_path = '/home/ubuntu/Kate/clip_alex/clean_img'
if not os.path.exists(extract_folder_path):
    os.makedirs(extract_folder_path)


with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder_path)

print(f'Файлы распакованы в: {extract_folder_path}')


for root, dirs, files in os.walk(extract_folder_path):
    print(f'В каталоге {root} найдено {len(dirs)} подкаталогов и {len(files)} файлов')

In [12]:
import os

# Path to the directory where you want to remove .ipynb_checkpoints
dir_path = '/home/ubuntu/Kate/clip_alex/top_top'


def remove_checkpoints(path):
    for root, dirs, files in os.walk(path):
        if '.ipynb_checkpoints' in dirs:
            checkpoint_path = os.path.join(root, '.ipynb_checkpoints')
            print(f"Removing {checkpoint_path}")
            os.rmdir(checkpoint_path)

remove_checkpoints(dir_path)

In [4]:
import os
dir_path = '/home/ubuntu/Kate/clip_alex/clean_img/robbi'

# Get list of directories
directories = os.listdir(dir_path)

# Separate and exclude '.ipynb_checkpoints' if it exists
sorted_directories = sorted(directories, key=lambda x: int(x))

print("Sorted directories including .ipynb_checkpoints at the beginning:")
print(sorted_directories)


Sorted directories including .ipynb_checkpoints at the beginning:
['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35']


## Model

In [11]:
import os
from random import shuffle
import torch
import clip
from PIL import Image
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

filenames = []
target_labels = []
dir_path = '/home/ubuntu/Kate/clip_alex/clean_img/robbi'
directories = os.listdir(dir_path)

sorted_directories = sorted(directories, key=lambda x: int(x))

for idx, folder in enumerate(sorted_directories):
    folder_path = os.path.join(dir_path, folder)
    for filename in os.listdir(folder_path):
        if not filename.endswith(('.jpg', '.jpeg', '.png')):
            continue
        filenames.append(os.path.join(folder_path, filename))
        target_labels.append(idx)

print("Target Labels:", target_labels[:6])
print("Filenames:", filenames[:6])

# Текстовые описания каждой уникальной игрушки
descriptions = [
    "The figure features a white and yellow color scheme, with yellow accents on its ears, arms, and boots. It has a transparent helmet with a hat on top, gloves draped like a scarf on the chest, and warm, padded boots on its feet.",
    "This figure has a red bird highlight on the helmet, sleek black glossy, red patterns on the sleeves and legs, Yin-Yang on the body",
    "The robot features a transparent head with a triangle and a ball in the center, creating focal point. Its color appears to change depending on the background and lighting, leading to varying light distortions. The body has a blue gradient towards the bottom.",
    "The robot has a glossy black finish with red accents and displays a 'B' logo on its chest.",
    "This robot is with a solid dark purple and burgundy color scheme, featuring sharp black protrusions extending from the sleeves. The body is adorned with a prominent emblem. The overall design is sleek with defined edges, emphasizing the sharp contrasts between the dark purple, burgundy, and black elements",
    "This figure is transparent with a gradient of pastel rainbow colors. Its helmet is filled with small colorful balls.",
    "The robot is white with a metallic finish on its helmet. It is depicted next to a sleek, futuristic bicycle.",
    "This robot is purple with a metallic sheen and is positioned beside a matching futuristic chair.",
    "The figure is transparent orange with intricate flower patterns embedded throughout its body. It has a glossy finish and the branding 'CREED 1760' is prominently displayed on its chest.",
    "This robot is made of transparent material with colored geometric patterns, including squares and rectangles, in primary colors such as red, blue, and yellow on the head and chest.",
    "The robot has a shiny golden finish with intricate floral patterns in various colors on its body and ears.",
    "This robot is predominantly green with white lines resembling tactical sports diagrams. It features a reflective helmet with 'LALIGA' written on it.",
    "This figure is dark green with black accents and yellow details. It has the 'Aston Martin Shanghai' logo on its chest.",
    "The robot is porcelain white body with blue Gzhel patterns on the sleeves, ear, shoes, and bottom of the helmet, emblem on the body.",
    "This pink robot is made of transparent, iridescent material with a pink heart emblem on its chest and helmet. Not yellow",
    "The robot is transparent body with a yelow hue, a pink heart inside the helmet, text on the body",
    "This white robot with a translucent tint, snow patterns, transparent helmet inside, various small shiny objects, white ear tips.",
    "The grey robot is metallic silver with black accents and features the Lamborghini logo on its chest.",
    "The robot is made of transparent purple material with a heart emblem on its chest and helmet,without triangle and pink",
    "The robot is completely gold with a highly reflective, shiny surface.",
    "The robot features a pink transparent head and ears, with a silver body. It has a pink heart emblem on its chest.",
    "This figure is red-orange, black helmet, red ears, matte.",
    "The robot has an iridescent, transparent body with a blue and red helmet and sole. blue NBA jersey with red stripes. It features NBA branding and a basketball jersey design.",
    "This figure is covered entirely with various NBA team stickers, creating a collage effect across its body and ears.",
    "The robot is transparent with yellow ears, arms, and shoes. It has red accents and a 'Peko' logo on its chest.",
    "The robot is black with a shiny silver helmet and a red visor line. It has a minimalist, futuristic design.",
    "This figure is entirely black with intricate patterns on its ears and chest. The body is matte and smooth, shiny patterns on the body and ears",
    "The robot features a gradient of colors, with red ears, a yellow body, turquoise legs, and purple shoes.",
    "The robot has a deep blue transparent body and helmet, giving it a sleek and glossy appearance.",
    "This figure is white with red and silver detailing. It has a clear, reflective helmet.",
    "The robot has a metallic blue reflective body and helmet. It features a clear visor with white patterns and the word 'Swift' on its chest. Not purple!",
    "The robot is transparent with pink ears, arms, and boots. It features various white line drawings and symbols on its body.Has no heart inside the helmet!not yellow",
    "The robot has an iridescent, rainbow-like color with predominant white and pink shades that change depending on the lighting, the transparent visor, a pink and green stripe on the helmet, and the 'Wilson' inscription on the chest.",
    "This figure is brown or bronze, made of wood.",
    "The robot is covered in long, white fur with a reflective visor."
]

# Привязываем эти описания к индексам, 1 описание соответстует 1 папке 
labels = descriptions

# Загрузка модели CLIP
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-L/14@336px", device=device)

# Функция предсказания top-k меток
def predict_top_k(model, image_path, texts, k=5):
    image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
    text = clip.tokenize(texts).to(device)
    
    with torch.no_grad():
        image_features = model.encode_image(image).float()
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features = model.encode_text(text).float()
        text_features /= text_features.norm(dim=-1, keepdim=True)

    text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    top_k_indices = text_probs.cpu().topk(k, dim=-1).indices[0]
    
    return top_k_indices, text_probs[0, top_k_indices]

all_top_k_indices = []
all_true_labels = []

for image_path, true_label in zip(filenames, target_labels):
    top_k_indices, _ = predict_top_k(model, image_path, labels, k=5)
    all_top_k_indices.append(top_k_indices)
    all_true_labels.append(true_label)

top_1_predictions = [indices[0].item() for indices in all_top_k_indices]

print(classification_report(all_true_labels, top_1_predictions, target_names=labels))


Target Labels: [0, 0, 0, 0, 0, 0]
Filenames: ['/home/ubuntu/Kate/clip_alex/clean_img/robbi/1/ROBBi x CHENPENG BUTTERMOON Co-brand Figures 1000%RED@鲨鱼小饱饱 with ROBBi#ROBBi #chenpeng #figures #ROBBiGLOBAL.jpg', '/home/ubuntu/Kate/clip_alex/clean_img/robbi/1/ROBBi x CHENPENG BUTTERMOON Co-brand Figures 1000%RED@鲨鱼小饱饱 with ROBBi#ROBBi #chenpeng #figures #ROBBiGLOBAL (3).jpg', '/home/ubuntu/Kate/clip_alex/clean_img/robbi/1/ROBBi x CHENPENG BUTTERMOON Mid-Autumn Festival theme co-brand series, the beauty of modern fashion, cross-bo.jpg', '/home/ubuntu/Kate/clip_alex/clean_img/robbi/1/ROBBi x CHENPENG BUTTERMOON Co-brand Figures 400%RED@玩具女侠 with ROBBi#ROBBi #chenpeng #figures #ROBBiGLOBAL (1).jpg', '/home/ubuntu/Kate/clip_alex/clean_img/robbi/1/ROBBi x CHENPENG BUTTERMOON Co-brand Figures 1000%RED@海豚Jessy有点辣 with ROBBi#ROBBi #chenpeng #figures #ROBBiGLOBAL (1).jpg', '/home/ubuntu/Kate/clip_alex/clean_img/robbi/1/ROBBi x CHENPENG BUTTERMOON Co-brand Figures 400%RED@玩具女侠 with ROBBi#ROBBi #chenp

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-L/14@336px", device=device)

# Overall

In [18]:
import os
from random import shuffle
import torch
import clip
from PIL import Image
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
import pandas as pd

top_5_correct = []
for true_label, top_k in zip(all_true_labels, all_top_k_indices):
    top_5_correct.append(true_label in top_k)

total_images_per_category = Counter(all_true_labels)

# Calculate Top-5 accuracy per category
top_5_accuracy_per_category = {}
for label in total_images_per_category.keys():
    correct_predictions = sum(1 for i, correct in enumerate(top_5_correct) if all_true_labels[i] == label and correct)
    top_5_accuracy_per_category[label] = round((correct_predictions / total_images_per_category[label]) * 100, 2)

# Calculate overall Top-5 accuracy
overall_top_5_accuracy = round((sum(top_5_correct) / len(all_true_labels)) * 100, 2)

top_5_accuracy_table = []
for label, accuracy in top_5_accuracy_per_category.items():
    top_5_accuracy_table.append([label, labels[label], accuracy])

top_5_accuracy_df = pd.DataFrame(top_5_accuracy_table, columns=['Label Index', 'Label', 'Top-5 Accuracy'])
print(top_5_accuracy_df.sort_values(by="Top-5 Accuracy"))

print(f"\nOverall Top-5 Accuracy: {overall_top_5_accuracy}%")
large = top_5_accuracy_df.sort_values(by="Top-5 Accuracy")["Top-5 Accuracy"].values.astype(float)
top_5_accuracy_df.to_csv("top_5_accuracy_df1.csv", index=False)

    Label Index                                              Label  \
26           26  This figure is entirely black with intricate p...   
7             7  This robot is purple with a metallic sheen and...   
9             9  This robot is made of transparent material wit...   
6             6  The robot is white with a metallic finish on i...   
29           29  This figure is white with red and silver detai...   
0             0  The figure features a white and yellow color s...   
17           17  The grey robot is metallic silver with black a...   
8             8  The figure is transparent orange with intricat...   
33           33      This figure is brown or bronze, made of wood.   
31           31  The robot is transparent with pink ears, arms,...   
13           13  The robot is porcelain white body with blue Gz...   
21           21  This figure is red-orange, black helmet, red e...   
2             2  The robot features a transparent head with a t...   
10           10  The

In [74]:
import os
from random import shuffle
import torch
import clip
from PIL import Image
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
import pandas as pd

top_5_correct = []
for true_label, top_k in zip(all_true_labels, all_top_k_indices):
    top_5_correct.append(true_label in top_k)

total_images_per_category = Counter(all_true_labels)

# Calculate Top-5 accuracy per category
top_5_accuracy_per_category = {}
for label in total_images_per_category.keys():
    correct_predictions = sum(1 for i, correct in enumerate(top_5_correct) if all_true_labels[i] == label and correct)
    top_5_accuracy_per_category[label] = round((correct_predictions / total_images_per_category[label]) * 100, 2)

# Calculate overall Top-5 accuracy
overall_top_5_accuracy = round((sum(top_5_correct) / len(all_true_labels)) * 100, 2)

top_5_accuracy_table = []
for label, accuracy in top_5_accuracy_per_category.items():
    top_5_accuracy_table.append([label, labels[label], accuracy])

top_5_accuracy_df = pd.DataFrame(top_5_accuracy_table, columns=['Label Index', 'Label', 'Top-5 Accuracy'])
print(top_5_accuracy_df.sort_values(by="Top-5 Accuracy"))

print(f"\nOverall Top-5 Accuracy: {overall_top_5_accuracy}%")
large = top_5_accuracy_df.sort_values(by="Top-5 Accuracy")["Top-5 Accuracy"].values.astype(float)
top_5_accuracy_df.to_csv("top_5_accuracy_df1.csv", index=False)

In [55]:
top_5_correct = []
for true_label, top_k in zip(all_true_labels, all_top_k_indices):
    top_5_correct.append(true_label in top_k)

total_images_per_category = Counter(all_true_labels)

# Calculate Top-5 accuracy per category
top_5_accuracy_per_category = {}
for label in total_images_per_category.keys():
    correct_predictions = sum(1 for i, correct in enumerate(top_5_correct) if all_true_labels[i] == label and correct)
    top_5_accuracy_per_category[label] = round((correct_predictions / total_images_per_category[label]) * 100, 2)

# Calculate overall Top-5 accuracy
overall_top_5_accuracy = round((sum(top_5_correct) / len(all_true_labels)) * 100, 2)

top_5_accuracy_table = []
for label, accuracy in top_5_accuracy_per_category.items():
    top_5_accuracy_table.append([label, labels[label], accuracy])

top_5_accuracy_df = pd.DataFrame(top_5_accuracy_table, columns=['Label Index', 'Label', 'Top-5 Accuracy'])
print(top_5_accuracy_df.sort_values(by="Top-5 Accuracy"))

print(f"\nOverall Top-5 Accuracy: {overall_top_5_accuracy}%")
base = top_5_accuracy_df.sort_values(by="Top-5 Accuracy")["Top-5 Accuracy"].values.astype(float)
top_5_accuracy_df.to_csv("top_5_accuracy_df1.csv", index=False)

In [75]:
large

In [56]:
import matplotlib.pyplot as plt
plt.plot(large, label = 'large')
plt.plot(base, label = 'base')
plt.legend()
plt.show()

# Топ 5 ошибок для каждой категории

In [63]:
import pandas as pd

mistake_df = pd.DataFrame(columns=['True Category', 'Mistaken Category', 'Count'])

mistake_records = []

for true_label, top_k_indices in zip(all_true_labels, all_top_k_indices):
    for idx in top_k_indices[1:]:  
        mistake_records.append((labels[true_label], labels[idx.item()]))

mistake_df = pd.DataFrame(mistake_records, columns=['True Category', 'Mistaken Category'])
mistake_summary = mistake_df.groupby(['True Category', 'Mistaken Category']).size().reset_index(name='Count')

# Функция для отображения и сохранения 5 наиболее частых ошибочных категорий для каждой категории
def display_and_save_top_5_mistakes_table(mistake_summary, labels):
    top_5_mistakes_dict = {}
    for label in labels:
        mistakes = mistake_summary[mistake_summary['True Category'] == label]
        top_5_mistakes = mistakes.sort_values(by='Count', ascending=False).head(5)
        top_5_mistakes_dict[label] = top_5_mistakes
    
    final_table = pd.concat(top_5_mistakes_dict.values()).reset_index(drop=True)
    
    final_table.to_csv("final_table1.csv", index=False)
    
    return final_table

final_table = display_and_save_top_5_mistakes_table(mistake_summary, labels)

In [70]:
descriptions[21]

In [68]:
final_table[final_table["True Category"] == descriptions[21]]["True Category"].values

In [69]:
final_table[final_table["True Category"] == descriptions[21]]["Mistaken Category"].values

# Лирика

In [29]:
from collections import defaultdict

errors = defaultdict(list)
for true_label, top_k_indices in zip(all_true_labels, all_top_k_indices):
    for idx in top_k_indices:
        if idx != true_label:
            errors[true_label].append((idx, top_k_indices.tolist().count(idx)))

for true_label, error_list in errors.items():
    sorted_errors = sorted(error_list, key=lambda x: x[1], reverse=True)[:5]
    print(f"Top 5 errors for category {true_label} ({labels[true_label]}):")
    for error, count in sorted_errors:
        print(f"  Predicted {error} ({labels[error]}): {count} times")

all_errors = []
for error_list in errors.values():
    all_errors.extend(error_list)

sorted_all_errors = sorted(all_errors, key=lambda x: x[1], reverse=True)[:5]
print("Top 5 errors overall:")
for error, count in sorted_all_errors:
    print(f"  Predicted {error} ({labels[error]}): {count} times")

In [ ]:
!git clone https://github.com/Joeclinton1/google-images-download.git
!cd google-images-download && python setup.py install
!rm -rf downloads/*
!googleimagesdownload -k "white Dress" -l 50
!googleimagesdownload -k "beige Dress" -l 50
!googleimagesdownload -k "yellow Dress" -l 50
!googleimagesdownload -k "orange Dress" -l 50
!googleimagesdownload -k "green Dress" -l 50
!googleimagesdownload -k "brown Dress" -l 50
!googleimagesdownload -k "red Dress" -l 50
!googleimagesdownload -k "pink Dress" -l 50
!googleimagesdownload -k "gray Dress" -l 50
!googleimagesdownload -k "purple Dress" -l 50
!googleimagesdownload -k "black Dress" -l 50
!googleimagesdownload -k "blue Dress" -l 50
!googleimagesdownload -k "light blue Dress" -l 50

In [67]:
for filename in filenames_test:
  image_labels(filename, labels = labels)

Overall Top-5 Accuracy

In [60]:
## CLIP package installation

In [1]:
%%capture
!conda install --yes -c pytorch pytorch=1.7.1 torchvision cudatoolkit=11.0
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

In [3]:
import torch
import clip
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)
#model is CLIP model, preproccess is an image transformation for the model